In [1]:
# Class5_DemographicAnalysis

In [2]:
from shapely.geometry import box
import matplotlib.pyplot as plt
from contextily import add_basemap
import contextily as cx
from shapely.geometry import Point, Polygon
from shapely.ops import transform
from pyproj import Transformer
import geopandas as gpd
import numpy as np
import time
import requests
import pandas as pd

In [3]:
import geopandas as gpd
from shapely.geometry import box
import matplotlib.pyplot as plt
from contextily import add_basemap
import contextily as cx

# 选择适合的投影坐标系统
projected_crs = '3857'  # Web Mercator 投影

parks_detail = gpd.read_file('park_details_isochrones_10_walking_gdf.geojson')
parks_detail = parks_detail.to_crs(epsg=projected_crs)  

parks_detail.columns

Index(['OBJECTID', 'NAME', 'PMA_NAME', 'ADDRESS', 'PIN', 'SUBPARCEL',
       'TOTAL_AREA', 'OWNER', 'LEASE', 'MAINT', 'TYPE', 'ACQ_DATE', 'NAMEFLAG',
       'REVIEW_DATE', 'AMWOID', 'SDQL', 'SE_ANNO_CAD_DATA', 'GIS_CRT_DT',
       'GIS_EDT_DT', 'GlobalID', 'centroid', 'isochrones', 'geometry'],
      dtype='object')

In [4]:
from shapely import wkt
parks_detail['isochrones'] = parks_detail['isochrones'].apply(wkt.loads)
parks_detail['centroid'] = parks_detail['centroid'].apply(wkt.loads)
selected_columns = parks_detail[['OBJECTID', 'isochrones','centroid']]
parks_isochrones = gpd.GeoDataFrame(selected_columns, geometry='isochrones')
parks_isochrones.set_crs(epsg=projected_crs,inplace=True)
parks_isochrones = parks_isochrones.to_crs(epsg=projected_crs)
parks_isochrones.head()

,OBJECTID,isochrones,centroid
0,660345,"POLYGON ((-13623319.47 6031677.956, -13623585....",POINT (-13622744.893154 6023634.083662)
1,659362,"POLYGON ((-13623401.623 6038683.571, -13623512...",POINT (-13622815.0938 6023678.225553)
2,659360,"POLYGON ((-13616262.816 6035189.128, -13616374...",POINT (-13622787.819986 6023732.238333)
3,660174,"POLYGON ((-13611805.027 6028358.78, -13611916....",POINT (-13622852.340365 6023754.704958)
4,659359,"POLYGON ((-13619588.486 6033210.681, -13619699...",POINT (-13622926.851761 6023768.188662)


In [5]:
# 加载数据
shapefile_path = 'census_2020.shp'
census_gdf = gpd.read_file(shapefile_path)
gdf_proj = census_gdf.to_crs(epsg=projected_crs)
# 列名映射字典
column_renames = {
    'TOTAL_POPU':'TOTAL_POPULATION',
    'Children_u':'CHILDREN_UNDER_5',
    'Older_Adul':'OLDER_ADULTS_65',
    'LESS_THAN_':'LESS_THAN_HIGH_SCHOOL',
    'BACHELOR_H':'BACHELOR_HIGHER',
    'TOTAL_HO_1':'TOTAL_HOUSING_UNITS',
    'LOW_DENSIT':'LOW_DENSITY',
    'MEDIUM_DEN':'MEDIUM_DENSITY',
    'UNIT_50_MO':'HIGHER_DENSITY',
    'OVERCROWDE':'OVERCROWDED',
    'POP_20_643':'EMPLOYED',
    'PER_CAPITA':'PER_CAPITA_INCOME',
    'NOT_HISPAN': 'WHITE_PEOPLE',
    'NOTHISPLAT': 'BLACK_PEOPLE',
    'NOTHISPL_1': 'INDIAN_ALASKA_PEOPLE',
    'NOTHISPL_2': 'ASIAN_PEOPLE',
    'NOTHISPL_3': 'HAWAIIAN_PACIFIC_PEOPLE',
    'NOTHISPL_4': 'OTHER_RACE_PEOPLE',
    'NOTHISPL_5': 'TWO_OR_MORE_RACES_PEOPLE',
    'HISPANIC_O': 'HISPANIC_PEOPLE',
    'PEOPLE_OF_': 'PEOPLE_OF_COLOR',
    'Median_Age': 'MEDIAN_AGE'
}

# 修改列名
gdf_proj = gdf_proj.rename(columns=column_renames)

# 计算人口密度
gdf_proj['POP_DENSITY'] = gdf_proj['TOTAL_POPULATION']  / (gdf_proj['ACRES_LAND']+ gdf_proj['ACRES_WATE']) * 100
# 计算就业率
gdf_proj['EMPLOYED_PERCENTAGE'] = gdf_proj['EMPLOYED'] / gdf_proj['TOTAL_POPULATION'] * 100
# 计算5岁以下的概率 
gdf_proj['UNDER5_PERCENTAGE'] = gdf_proj['CHILDREN_UNDER_5'] / gdf_proj['TOTAL_POPULATION'] * 100
# 65岁以上
gdf_proj['65_PERCENTAGE'] = gdf_proj['OLDER_ADULTS_65'] / gdf_proj['TOTAL_POPULATION'] * 100
# 有色人种的比例
gdf_proj['PEOPLE_OF_COLOR_PERCENTAGE'] = gdf_proj['PEOPLE_OF_COLOR'] / gdf_proj['TOTAL_POPULATION'] * 100
# 贫穷人口比例
gdf_proj['POVERTY_PERCENTAGE'] = (gdf_proj['INC_POV_RA'] + gdf_proj['INC_POV__1']) / gdf_proj['TOTAL_POPULATION'] * 100

In [6]:
geojson_path = 'SeattleDemographic_Block.geojson'
# 保存为新的 GeoJSON 文件
gdf_proj.to_file(geojson_path, driver='GeoJSON')
print(f"Geojsonfile has been renamed and saved to {geojson_path}")

Geojsonfile has been renamed and saved to SeattleDemographic_Block.geojson


In [7]:
# 空间连接
# 使用 'intersects' 或 'within' 进行空间连接，根据你的数据特点选择适合的连接方式
joined = gpd.sjoin(gdf_proj,parks_isochrones,how = 'inner',predicate = 'intersects')
joined.head()

,GEOID_20,ACRES_LAND,ACRES_WATE,TRACT,BG,TRBG,BG_NAME,TRACT_LABE,C_DISTRICT,VILLNUMB,...,geometry,POP_DENSITY,EMPLOYED_PERCENTAGE,UNDER5_PERCENTAGE,65_PERCENTAGE,PEOPLE_OF_COLOR_PERCENTAGE,POVERTY_PERCENTAGE,index_right,OBJECTID,centroid
0,530330105022,93.418437,0.0,10502,2,10502.2,Block Group 2,105.02,1,0.0,...,"POLYGON ((-13624397.027 6033490.507, -13624398...",2236.175294,57.683102,4.164672,17.041647,26.089038,9.430349,1143,659764,POINT (-13613386.760669 6028805.610623)
0,530330105022,93.418437,0.0,10502,2,10502.2,Block Group 2,105.02,1,0.0,...,"POLYGON ((-13624397.027 6033490.507, -13624398...",2236.175294,57.683102,4.164672,17.041647,26.089038,9.430349,1267,658864,POINT (-13616658.5733 6034419.948559)
0,530330105022,93.418437,0.0,10502,2,10502.2,Block Group 2,105.02,1,0.0,...,"POLYGON ((-13624397.027 6033490.507, -13624398...",2236.175294,57.683102,4.164672,17.041647,26.089038,9.430349,1071,660766,POINT (-13610249.574539 6028373.903856)
0,530330105022,93.418437,0.0,10502,2,10502.2,Block Group 2,105.02,1,0.0,...,"POLYGON ((-13624397.027 6033490.507, -13624398...",2236.175294,57.683102,4.164672,17.041647,26.089038,9.430349,1955,660623,POINT (-13615460.814991 6048239.588305)
0,530330105022,93.418437,0.0,10502,2,10502.2,Block Group 2,105.02,1,0.0,...,"POLYGON ((-13624397.027 6033490.507, -13624398...",2236.175294,57.683102,4.164672,17.041647,26.089038,9.430349,1715,661128,POINT (-13617206.732897 6040336.247379)


In [8]:
# 计算每个等时范围内的总人口密度和/或平均人口密度
# 假设 'density' 是人口密度的列名，'OBJECTID' 是等时范围的唯一标识符列名

# 计算平均人口密度
average_density = joined.groupby('OBJECTID').agg({
    'POP_DENSITY': 'mean',  # 计算平均人口密度
    'EMPLOYED_PERCENTAGE': 'mean',     
    'UNDER5_PERCENTAGE': 'mean' ,
    '65_PERCENTAGE' :'mean',
    'PEOPLE_OF_COLOR_PERCENTAGE' :'mean',
    'POVERTY_PERCENTAGE' : 'mean'
     
}).reset_index()
average_density.columns = ['OBJECTID', 'POP_DENSITY',
    'EMPLOYED_PERCENTAGE',     
    'UNDER5_PERCENTAGE',
    '65_PERCENTAGE' ,
    'PEOPLE_OF_COLOR_PERCENTAGE',
    'POVERTY_PERCENTAGE']

# 合并结果到等时矩阵数据
parks_isochrones = parks_isochrones.merge(average_density, on='OBJECTID', how='left')

In [9]:
parks_isochrones.to_csv('parks_isochrones_with_demographic.csv')

In [10]:
parks_isochrones.head()

,OBJECTID,isochrones,centroid,POP_DENSITY,EMPLOYED_PERCENTAGE,UNDER5_PERCENTAGE,65_PERCENTAGE,PEOPLE_OF_COLOR_PERCENTAGE,POVERTY_PERCENTAGE
0,660345,"POLYGON ((-13623319.47 6031677.956, -13623585....",POINT (-13622744.893154 6023634.083662),1340.473015,53.224039,7.445560,14.533173,28.095383,8.705330
1,659362,"POLYGON ((-13623401.623 6038683.571, -13623512...",POINT (-13622815.0938 6023678.225553),1090.873153,55.335690,5.004556,13.980434,21.342277,2.651710
2,659360,"POLYGON ((-13616262.816 6035189.128, -13616374...",POINT (-13622787.819986 6023732.238333),748.194085,51.364259,3.178540,20.553536,72.919848,7.432395
3,660174,"POLYGON ((-13611805.027 6028358.78, -13611916....",POINT (-13622852.340365 6023754.704958),1138.385089,44.467581,6.452216,15.853316,75.931649,15.587020
4,659359,"POLYGON ((-13619588.486 6033210.681, -13619699...",POINT (-13622926.851761 6023768.188662),187.901788,66.224703,4.815073,11.304955,29.867411,5.094208
